<a href="https://colab.research.google.com/github/moseskim/bert_nlp/blob/main/section_2/03_simple_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 BERT 구현

훈련 완료 모델을 사용해 문장의 일부를 예측하고, 2개의 문장이 연속되어 있는지 판정합니다.

## 라이브러리 설치

PyTorch-Transformers 및 필요한 라이브러리를 설치합니다.

In [ ]:
!pip install folium==0.2.1
!pip install urllib3==1.25.11
!pip install pytorch-transformers==1.2.0

## 문장 일부 예측

문자에서 일부 단어를 MASK하고, BERT 모델을 사용해 그 단어를 예측합니다.

In [ ]:
import torch
from pytorch_transformers import BertForMaskedLM
from pytorch_transformers import BertTokenizer


text = "[CLS] I played baseball with my friends at school yesterday [SEP]"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
words = tokenizer.tokenize(text)
print(words)

문장 일부를 MASK합니다.

In [ ]:
msk_idx = 3
words[msk_idx] = "[MASK]"  # 단어를 [MASK]로 치환한다
print(words)

단어를 대응하는 인덱스로 변환합니다.

In [ ]:
word_ids = tokenizer.convert_tokens_to_ids(words)  # 단어를 인덱스로 변환
word_tensor = torch.tensor([word_ids])  # 텐서로 변환
print(word_tensor)

BERT 모델을 사용해 예측을 수행합니다.

In [ ]:
msk_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
msk_model.cuda()  # GPU 대응
msk_model.eval()

x = word_tensor.cuda()  # GPU 대응
y = msk_model(x)  # 예측
result = y[0]
print(result.size())  # 결과의 형태

_, max_ids = torch.topk(result[0][msk_idx], k=5)  # 가장 큰 5개의 값
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())  # 인덱스를 단어로 변환
print(result_words)

## 문장이 연속되어 있는지 판정

BERT 모델을 사용해 2개의 문장이 연속되어 있는지 판정합니다.  
다음의 함수 `show_continuity`에서는 2개 문장의 연속성을 판정하고 표시합니다.  

In [ ]:
from pytorch_transformers import BertForNextSentencePrediction

def show_continuity(text, seg_ids):
    words = tokenizer.tokenize(text)
    word_ids = tokenizer.convert_tokens_to_ids(words)  # 단어를 인덱스로 변환
    word_tensor = torch.tensor([word_ids])  # 텐서로 변환

    seg_tensor = torch.tensor([seg_ids])

    nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
    nsp_model.cuda()  # GPU 대응
    nsp_model.eval()

    x = word_tensor.cuda()  # GPU 대응
    s = seg_tensor.cuda()  # GPU 대응

    y = nsp_model(x, s)  # 예측
    result = torch.softmax(y[0], dim=1)
    print(result)  # Softmax로 확률로
    print(str(result[0][0].item()*100) + "%의 확률로 연속됩니다.")

`show_continuity` 함수에 자연스럽게 이어지는 2개의 문장을 전달합니다.

In [ ]:
text = "[CLS] What is baseball ? [SEP] It is a game of hitting the ball with the bat [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1]  # 0: 앞 문장의 단어, 1: 뒷 문장의 단어
show_continuity(text, seg_ids)

`show_continuity` 함수에 자연스럽게 이어지지 않는 2개의 문장을 전달합니다.

In [ ]:
text = "[CLS] What is baseball ? [SEP] This food is made with flour and milk [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]  # 0: 앞 문장의 단어, 1: 뒷 문장의 단어
show_continuity(text, seg_ids)